In [2]:
import requests
import pandas as pd

In [3]:
KEY = 'F54F3D1VUSH0TQJF'
STOCK = 'IBM'
# can get your own at alpha vantage for free. 

In [2]:
options_url = 'https://www.alphavantage.co/query?function=HISTORICAL_OPTIONS&symbol='+STOCK+'&apikey='+KEY
r = requests.get(options_url)
data = r.json()
data = data['data']
pd.DataFrame(data).to_csv('options.csv', index=False)
options = pd.read_csv('options.csv')

In [23]:
options.head()

,contractID,symbol,expiration,strike,type,last,mark,bid,bid_size,ask,ask_size,volume,open_interest,date,implied_volatility,delta,gamma,theta,vega,rho
0,IBM240621C00060000,IBM,2024-06-21,60.0,call,123.1,110.58,108.70,37,112.45,45,0,0,2024-06-18,3.47280,0.99975,0.00002,-0.06284,0.00015,0.00493
1,IBM240621P00060000,IBM,2024-06-21,60.0,put,0.0,0.01,0.00,0,1.03,48,0,34,2024-06-18,3.75114,-0.00059,0.00004,-0.07312,0.00032,-0.00001
2,IBM240621C00065000,IBM,2024-06-21,65.0,call,80.0,105.53,103.65,37,107.40,36,0,0,2024-06-18,2.48326,1.00000,0.00000,-0.03463,0.00000,0.00534
3,IBM240621P00065000,IBM,2024-06-21,65.0,put,0.0,0.01,0.00,0,1.03,52,0,30,2024-06-18,3.47478,-0.00064,0.00004,-0.07277,0.00034,-0.00001
4,IBM240621C00070000,IBM,2024-06-21,70.0,call,52.8,100.58,98.70,37,102.45,49,0,0,2024-06-18,2.36562,1.00000,0.00000,-0.03729,0.00000,0.00575


In [4]:
prices_url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol='+STOCK+'&apikey=F54F3D1VUSH0TQJF'
r = requests.get(prices_url)
data = r.json()
data.keys()
data = data['Time Series (Daily)']
pd.DataFrame(data).to_csv('prices.csv', index=False)
prices = pd.read_csv('prices.csv')
prices = prices.T

KeyError: 'data'

In [12]:
prices.head()

,0,1,2,3,4
2024-06-18,170.00,170.75,168.380,170.55,3386442.0
2024-06-17,168.76,169.72,167.500,169.50,3239815.0
2024-06-14,168.29,169.47,167.230,169.21,2777717.0
2024-06-13,169.01,169.59,168.335,169.12,3525717.0
2024-06-12,171.35,172.47,168.101,169.00,3522698.0


In [13]:
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from scipy.stats import norm

In [ ]:
''' Assuming these options are European, not American. (Can use the model made by Robert Merton in further iterations)'''

In [14]:
TRANSACTION_COST = 0.001 
R = 0.05

In [32]:
def curr_price(date):
    return (prices.loc[date][0] + prices.loc[date][3])/2

In [33]:
from datetime import date,datetime
def days_till(start,exp):
    start = datetime.strptime(start, '%Y-%m-%d').date()

    exp = datetime.strptime(exp, '%Y-%m-%d').date()
    return (exp-start).days

In [38]:
def black_scholes(row):
    S, K, r, T, sigma, option_type = curr_price(row['date']), row['strike'],R, days_till(row['date'],row['expiration']),row['implied_volatility'], row['type']
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    
    return round(price,2)

In [39]:
options['ideal_val'] = options.apply(black_scholes,axis=1) 

In [1]:
options['ideal_val'] = (options['ideal_val'] * TRANSACTION_COST) + options['ideal_val']

NameError: name 'options' is not defined

In [40]:
options

,contractID,symbol,expiration,strike,type,last,mark,bid,bid_size,ask,...,volume,open_interest,date,implied_volatility,delta,gamma,theta,vega,rho,ideal_val
0,IBM240621C00060000,IBM,2024-06-21,60.0,call,123.10,110.58,108.70,37,112.45,...,0,0,2024-06-18,3.47280,0.99975,0.00002,-0.06284,0.00015,0.00493,170.03
1,IBM240621P00060000,IBM,2024-06-21,60.0,put,0.00,0.01,0.00,0,1.03,...,0,34,2024-06-18,3.75114,-0.00059,0.00004,-0.07312,0.00032,-0.00001,51.54
2,IBM240621C00065000,IBM,2024-06-21,65.0,call,80.00,105.53,103.65,37,107.40,...,0,0,2024-06-18,2.48326,1.00000,0.00000,-0.03463,0.00000,0.00534,167.28
3,IBM240621P00065000,IBM,2024-06-21,65.0,put,0.00,0.01,0.00,0,1.03,...,0,30,2024-06-18,3.47478,-0.00064,0.00004,-0.07277,0.00034,-0.00001,55.69
4,IBM240621C00070000,IBM,2024-06-21,70.0,call,52.80,100.58,98.70,37,102.45,...,0,0,2024-06-18,2.36562,1.00000,0.00000,-0.03729,0.00000,0.00575,166.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1061,IBM260116P00270000,IBM,2026-01-16,270.0,put,0.00,99.50,97.00,37,102.00,...,0,0,2024-06-18,0.50625,-0.60684,0.00354,-0.02384,0.82460,-3.20912,0.00
1062,IBM260116C00280000,IBM,2026-01-16,280.0,call,1.25,1.51,1.22,20,1.80,...,0,322,2024-06-18,0.21327,0.08068,0.00327,-0.02817,0.32083,0.19363,170.27
1063,IBM260116P00280000,IBM,2026-01-16,280.0,put,0.00,109.50,107.05,37,111.95,...,0,0,2024-06-18,0.53567,-0.60805,0.00334,-0.02596,0.82390,-3.37033,0.00
1064,IBM260116C00290000,IBM,2026-01-16,290.0,call,0.98,1.05,0.80,37,1.30,...,0,749,2024-06-18,0.21022,0.05968,0.00263,-0.02178,0.25437,0.14430,170.27


In [5]:
options_url = 'https://www.alphavantage.co/query?function=HISTORICAL_OPTIONS&date=2008-01-01&symbol='+STOCK+'&apikey='+KEY
r = requests.get(options_url)
data = r.json()
data = data['data']
pd.DataFrame(data).to_csv('options.csv', index=False)
options = pd.read_csv('options.csv')

EmptyDataError: No columns to parse from file